In [1]:
### Imports
import streamlit as st
import os
import io
import json
import pandas as pd
from datetime import datetime
import chromadb

import langchain
# from langchain.cache import InMemoryCache
# langchain.llm_cache = InMemoryCache()
# from langchain.cache import SQLiteCache
# langchain.llm_cache = SQLiteCache(database_path="langchain.db")
from langchain.agents import initialize_agent, AgentType, load_tools
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool, Tool, tool
from langchain.callbacks.manager import AsyncCallbackManagerForToolRun, CallbackManagerForToolRun
from langchain.llms import OpenAI
from langchain.experimental.plan_and_execute import PlanAndExecute, load_agent_executor, load_chat_planner
from langchain import LLMMathChain
from langchain.vectorstores import Chroma
from langchain.agents.agent_toolkits import create_python_agent
from contextlib import redirect_stdout
from typing import Optional, Type
from langchain.document_loaders import TextLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

## Initialize

In [2]:
### Read runbook and create vector index
raw_documents = TextLoader('content/runbook.txt').load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
documents = text_splitter.split_documents(raw_documents)
runbook_vectors = Chroma.from_documents(documents, OpenAIEmbeddings())


In [3]:
# OpenAI Credentials
if not os.environ["OPENAI_API_KEY"]:
    openai_api_key = st.secrets["OPENAI_API_KEY"]
else:
    openai_api_key = os.environ["OPENAI_API_KEY"]


## Planner

In [84]:
def planner(myquestion):

    ### Bring in my controlling documents and the additonal template
    relevancy_cutoff = .8
    
    docs = runbook_vectors.similarity_search_with_relevance_scores(myquestion, k=3)

    mytasks = ""
    for x,v in docs:
        if mytasks == "":
            mytasks = str(x) + "\n\n"
            continue
        if v > relevancy_cutoff:
            mytasks = mytasks + str(x) + "\n\n"
            
    template="""
    You are a helpful chatbot that collects information from the user based upon the type of report they need to comnplete.
    Using the TEXT below, either answer the user's question or create a series of steps necessary to collect all the information necessary for the report type referenced in the  "PROMPT" below. 
    return a list of steps with each step labeled in the format "Step #:".
    If the user's request does not reference a report you understand, then DO NOT create any steps and respond with "Sorry, but I can't help you with that task."

    TEXT:
    {mytasks}

    PROMPT:
    {myquestion}
    """
        
    llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.0, verbose=True)

    chain = LLMChain(llm=llm, prompt=PromptTemplate.from_template(template))
    
    response = chain(inputs={"mytasks": mytasks,"myquestion": myquestion})

    return response



In [5]:
# prompt="What kind of reports can you support?"
prompt="I would like to create a burglary report."
# prompt="How do I create a crime scene report?"
# prompt="I would like to create a financial fraud report."

In [6]:
response = planner(prompt)
# print(response['text'])
steps = response['text'].split("\n")
df = pd.DataFrame(steps, columns=['steps'])
df.replace('(^\s+|\t+|\s+$)', '', regex=True, inplace=True)

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 1000)
print("PROMPT:",prompt+"\n")
print("STEPS:")
for index, row in df.iterrows():
    print(df['steps'].values[index])


PROMPT: I would like to create a burglary report.

STEPS:
Step 1: Ask the user for the location of the burglary. The user should provide the street address, city, and state. If the user cannot provide a street address, ask the user for a nearby landmark or cross streets.
Step 2: Ask the user for the date and time of the burglary, even if the time is just an estimate.
Step 3: Ask the user to list the property stolen. The user should provide a description of each item, when it was purchased, and an estimated value for each item.
Step 4: Calculate the total value of all items.
Step 5: Ask the user for their contact information, including name, address, and phone number.
Step 6: Ask the user to list the contact information for any witnesses, including name, address, phone numbers, and relationship to the user.


## Executor

In [7]:
def get_input(myprompt) -> str:
    print(myprompt)
    contents = input()
    return contents


# x=get_input("Enter list of items:")

def evaluator(myquestion, myanswer):

    template="""
    Evaluate the user's RESPONSE below with the QUESTION the user was asked. 
    If the RESPONSE DOES NOT adequately answer the QUESTION then respond with what information is missing from the answers.
   

    QUESTION:
    {myquestion}

    RESPONSE:
    {myanswer}
    """
    model="gpt-4"
#     model="gpt-3.5-turbo"
    llm = ChatOpenAI(model=model, temperature=0.0, verbose=False)
    ev_chain = LLMChain(llm=llm, prompt=PromptTemplate.from_template(template))
    response = ev_chain(inputs={"myanswer": myanswer,"myquestion": myquestion})

    return response



In [8]:
def executor(mydf):

    if mydf.size == 1:
        return mydf['steps'].values[0]
    if mydf.size == 0:
        return "I'm sorry but I can't help you. Use this tool to assist in collecting incident reports."
    for index, row in df.iterrows():
        userinput = get_input(df['steps'].values[index])
        if len(userinput)==0:
            break
        myinput = evaluator(df['steps'].values[index], userinput)
        df['response'][index] = userinput
        if "adequate" in myinput['text']:
            print(myinput['text'])
        else:
            userinput = get_input(myinput['text'])
            df['response'][index] = df['response'][index] + "\n" + userinput
        print()
    
df2 = pd.DataFrame()    
print(executor(df)) 

df.head(10)

Step 1: Ask the user for the location of the burglary. The user should provide the street address, city, and state. If the user cannot provide a street address, ask the user for a nearby landmark or cross streets.

None


,steps
0,"Step 1: Ask the user for the location of the burglary. The user should provide the street address, city, and state. If the user cannot provide a street address, ask the user for a nearby landmark or cross streets."
1,"Step 2: Ask the user for the date and time of the burglary, even if the time is just an estimate."
2,"Step 3: Ask the user to list the property stolen. The user should provide a description of each item, when it was purchased, and an estimated value for each item."
3,Step 4: Calculate the total value of all items.
4,"Step 5: Ask the user for their contact information, including name, address, and phone number."
5,"Step 6: Ask the user to list the contact information for any witnesses, including name, address, phone numbers, and relationship to the user."


## Lambda style

In [88]:
def new_lambda(myquestion, myanswer, myhistory="", myindex=0):

    if len(myhistory)== 0:
        myresults = planner(myanswer)
        # print(myresults)
        steps = myresults['text'].split("\n")
        # print(steps)
        mydf = pd.DataFrame(steps, columns=['steps'])
        mydf['response']=''
        mydf.replace('(^\s+|\t+|\s+$)', '', regex=True, inplace=True)
        if len(mydf.index) == 1:
            return myresults['text'], 'True', '', 0
        elif len(mydf.index) == 0:
            return "I'm sorry but I can't help you. Use this tool to assist in collecting incident reports.", 'True', '', 0
        else:
            result = mydf.to_json(orient="records")
            return mydf['steps'].values[0], 'False', result, 0
    else:
        mydf = pd.DataFrame.from_dict(json.loads(history), orient='columns')
        mydf['response'][myindex]=mydf['response'][myindex]+" "+myanswer
        evaluation = evaluator(myquestion, myanswer)
        myresults = evaluation['text']
        if "adequate" in myresults:
            myindex = myindex+1
            result = mydf.to_json(orient="records")
            return mydf['steps'].values[myindex], False, result, myindex
        else:
            result = mydf.to_json(orient="records")
            return myresults, False, result, myindex
        

In [77]:
myquestion = "how can I help you?"
myanswer = "What kind of reports can you support?"

answer, final, history, qnbr = new_lambda(myquestion, myanswer)
print(answer)
print(final)
print(history)
print(qnbr)

The system is currently configured to collect information on burglaries and crime scenes. However, the system can be expanded to include any type of report as long as instructions are provided in the runbook.
True

0


In [90]:
myquestion = "how can I help you?"
myanswer = "Help me complete a burglary report"

answer, final, history, qnbr = new_lambda(myquestion, myanswer)
print(answer)
print(final)
print(history)
print(qnbr)

Step 1: Ask the user for the location of the burglary. The user should provide the street address, city, and state. If the user cannot provide a street address, ask the user for a nearby landmark or cross streets.
False
[{"steps":"Step 1: Ask the user for the location of the burglary. The user should provide the street address, city, and state. If the user cannot provide a street address, ask the user for a nearby landmark or cross streets.","response":""},{"steps":"Step 2: Ask the user for the date and time of the burglary, even if the time is just an estimate.","response":""},{"steps":"Step 3: Ask the user to list the property stolen. The user should provide a description of each item, when it was purchased, and an estimated value for each item.","response":""},{"steps":"Step 4: Calculate the total value of all items.","response":""},{"steps":"Step 5: Ask the user for their contact information, including name, address, and phone number.","response":""},{"steps":"Step 6: Ask the user 

In [91]:
myquestion = answer
myanswer = "123 Main St."
myhistory = history
myqnbr = qnbr

answer, final, history, qnbr = new_lambda(myquestion, myanswer, myhistory, myqnbr)
print(answer)
print(final)
print(history)
print(qnbr)

The user did not provide the city and state of the burglary location.
False
[{"steps":"Step 1: Ask the user for the location of the burglary. The user should provide the street address, city, and state. If the user cannot provide a street address, ask the user for a nearby landmark or cross streets.","response":" 123 Main St."},{"steps":"Step 2: Ask the user for the date and time of the burglary, even if the time is just an estimate.","response":""},{"steps":"Step 3: Ask the user to list the property stolen. The user should provide a description of each item, when it was purchased, and an estimated value for each item.","response":""},{"steps":"Step 4: Calculate the total value of all items.","response":""},{"steps":"Step 5: Ask the user for their contact information, including name, address, and phone number.","response":""},{"steps":"Step 6: Ask the user to list the contact information for any witnesses, including name, address, phone numbers, and relationship to the user.","response

In [92]:
myquestion = answer
myanswer = "Buffalo, NY"
myhistory = history
myqnbr = qnbr

answer, final, history, qnbr = new_lambda(myquestion, myanswer, myhistory, myqnbr)
print(answer)
print(final)
print(history)
print(qnbr)

Step 2: Ask the user for the date and time of the burglary, even if the time is just an estimate.
False
[{"steps":"Step 1: Ask the user for the location of the burglary. The user should provide the street address, city, and state. If the user cannot provide a street address, ask the user for a nearby landmark or cross streets.","response":" 123 Main St. Buffalo, NY"},{"steps":"Step 2: Ask the user for the date and time of the burglary, even if the time is just an estimate.","response":""},{"steps":"Step 3: Ask the user to list the property stolen. The user should provide a description of each item, when it was purchased, and an estimated value for each item.","response":""},{"steps":"Step 4: Calculate the total value of all items.","response":""},{"steps":"Step 5: Ask the user for their contact information, including name, address, and phone number.","response":""},{"steps":"Step 6: Ask the user to list the contact information for any witnesses, including name, address, phone numbers, 

In [93]:
myquestion = answer
myanswer = "01/01/2023"
myhistory = history
myqnbr = qnbr

answer, final, history, qnbr = new_lambda(myquestion, myanswer, myhistory, myqnbr)
print(answer)
print(final)
print(history)
print(qnbr)

The time of the burglary is missing from the response.
False
[{"steps":"Step 1: Ask the user for the location of the burglary. The user should provide the street address, city, and state. If the user cannot provide a street address, ask the user for a nearby landmark or cross streets.","response":" 123 Main St. Buffalo, NY"},{"steps":"Step 2: Ask the user for the date and time of the burglary, even if the time is just an estimate.","response":" 01\/01\/2023"},{"steps":"Step 3: Ask the user to list the property stolen. The user should provide a description of each item, when it was purchased, and an estimated value for each item.","response":""},{"steps":"Step 4: Calculate the total value of all items.","response":""},{"steps":"Step 5: Ask the user for their contact information, including name, address, and phone number.","response":""},{"steps":"Step 6: Ask the user to list the contact information for any witnesses, including name, address, phone numbers, and relationship to the user."

In [94]:
myquestion = answer
myanswer = "12:00"
myhistory = history
myqnbr = qnbr

answer, final, history, qnbr = new_lambda(myquestion, myanswer, myhistory, myqnbr)
print(answer)
print(final)
print(history)
print(qnbr)

Step 3: Ask the user to list the property stolen. The user should provide a description of each item, when it was purchased, and an estimated value for each item.
False
[{"steps":"Step 1: Ask the user for the location of the burglary. The user should provide the street address, city, and state. If the user cannot provide a street address, ask the user for a nearby landmark or cross streets.","response":" 123 Main St. Buffalo, NY"},{"steps":"Step 2: Ask the user for the date and time of the burglary, even if the time is just an estimate.","response":" 01\/01\/2023 12:00"},{"steps":"Step 3: Ask the user to list the property stolen. The user should provide a description of each item, when it was purchased, and an estimated value for each item.","response":""},{"steps":"Step 4: Calculate the total value of all items.","response":""},{"steps":"Step 5: Ask the user for their contact information, including name, address, and phone number.","response":""},{"steps":"Step 6: Ask the user to list

In [95]:
myquestion = answer
myanswer = "1 gold watch with a value of $400. I don't know when it was purchased. It was a gift on Christmas, 2005."
myhistory = history
myqnbr = qnbr

answer, final, history, qnbr = new_lambda(myquestion, myanswer, myhistory, myqnbr)
print(answer)
print(final)
print(history)
print(qnbr)

The user did not provide a description of the item.
False
[{"steps":"Step 1: Ask the user for the location of the burglary. The user should provide the street address, city, and state. If the user cannot provide a street address, ask the user for a nearby landmark or cross streets.","response":" 123 Main St. Buffalo, NY"},{"steps":"Step 2: Ask the user for the date and time of the burglary, even if the time is just an estimate.","response":" 01\/01\/2023 12:00"},{"steps":"Step 3: Ask the user to list the property stolen. The user should provide a description of each item, when it was purchased, and an estimated value for each item.","response":" 1 gold watch with a value of $400. I don't know when it was purchased. It was a gift on Christmas, 2005."},{"steps":"Step 4: Calculate the total value of all items.","response":""},{"steps":"Step 5: Ask the user for their contact information, including name, address, and phone number.","response":""},{"steps":"Step 6: Ask the user to list the c

In [96]:
myquestion = answer
myanswer = "The watch is a Timex."
myhistory = history
myqnbr = qnbr

answer, final, history, qnbr = new_lambda(myquestion, myanswer, myhistory, myqnbr)
print(answer)
print(final)
print(history)
print(qnbr)

The user did not provide enough information about the watch. Details such as the color, size, features, condition, and model of the Timex watch are missing.
False
[{"steps":"Step 1: Ask the user for the location of the burglary. The user should provide the street address, city, and state. If the user cannot provide a street address, ask the user for a nearby landmark or cross streets.","response":" 123 Main St. Buffalo, NY"},{"steps":"Step 2: Ask the user for the date and time of the burglary, even if the time is just an estimate.","response":" 01\/01\/2023 12:00"},{"steps":"Step 3: Ask the user to list the property stolen. The user should provide a description of each item, when it was purchased, and an estimated value for each item.","response":" 1 gold watch with a value of $400. I don't know when it was purchased. It was a gift on Christmas, 2005. The watch is a Timex."},{"steps":"Step 4: Calculate the total value of all items.","response":""},{"steps":"Step 5: Ask the user for the

In [97]:
myquestion = answer
myanswer = "I cannot provide any more description of the watch."
myhistory = history
myqnbr = qnbr

answer, final, history, qnbr = new_lambda(myquestion, myanswer, myhistory, myqnbr)
print(answer)
print(final)
print(history)
print(qnbr)

The user did not provide the missing information about the color, size, features, condition, and model of the Timex watch.
False
[{"steps":"Step 1: Ask the user for the location of the burglary. The user should provide the street address, city, and state. If the user cannot provide a street address, ask the user for a nearby landmark or cross streets.","response":" 123 Main St. Buffalo, NY"},{"steps":"Step 2: Ask the user for the date and time of the burglary, even if the time is just an estimate.","response":" 01\/01\/2023 12:00"},{"steps":"Step 3: Ask the user to list the property stolen. The user should provide a description of each item, when it was purchased, and an estimated value for each item.","response":" 1 gold watch with a value of $400. I don't know when it was purchased. It was a gift on Christmas, 2005. The watch is a Timex. I cannot provide any more description of the watch."},{"steps":"Step 4: Calculate the total value of all items.","response":""},{"steps":"Step 5: As

In [98]:
myquestion = answer
myanswer = "Sinclair Timex Z-1. It is an analog style watch. The color is gold. it was in good condition."
myhistory = history
myqnbr = qnbr

answer, final, history, qnbr = new_lambda(myquestion, myanswer, myhistory, myqnbr)
print(answer)
print(final)
print(history)
print(qnbr)

The response is missing the size and specific features of the Sinclair Timex Z-1 watch.
False
[{"steps":"Step 1: Ask the user for the location of the burglary. The user should provide the street address, city, and state. If the user cannot provide a street address, ask the user for a nearby landmark or cross streets.","response":" 123 Main St. Buffalo, NY"},{"steps":"Step 2: Ask the user for the date and time of the burglary, even if the time is just an estimate.","response":" 01\/01\/2023 12:00"},{"steps":"Step 3: Ask the user to list the property stolen. The user should provide a description of each item, when it was purchased, and an estimated value for each item.","response":" 1 gold watch with a value of $400. I don't know when it was purchased. It was a gift on Christmas, 2005. The watch is a Timex. I cannot provide any more description of the watch. Sinclair Timex Z-1. It is an analog style watch. The color is gold. it was in good condition."},{"steps":"Step 4: Calculate the tot